In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
print(df_ratings.head())
df_books.head()

     user        isbn  rating
0  276725  034545104X     0.0
1  276726  0155061224     5.0
2  276727  0446520802     0.0
3  276729  052165615X     3.0
4  276729  0521795028     6.0


,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [5]:
cleaned_ratings = df_ratings[(df_ratings['isbn'].map(df_ratings['isbn'].value_counts() >= 100)) & df_ratings['user'].map(df_ratings['user'].value_counts() >= 200)]
filtered_rating_df = cleaned_ratings.merge(df_books[['title', 'isbn']], on = 'isbn', how = 'left')

filtered_rating_df = filtered_rating_df.drop_duplicates(['isbn', 'user'])
filtered_rating_df.drop(columns = 'isbn', inplace = True)

filtered_rating_df.dropna(inplace = True)
filtered_rating_df['title_code'] = filtered_rating_df['title'].astype('category').cat.codes

ratings = filtered_rating_df['rating'].to_numpy()
title_ids = filtered_rating_df['title_code'].to_numpy()
user_ids = filtered_rating_df['user'].to_numpy()

rating_matrix = csr_matrix((ratings, (title_ids, user_ids)))

In [6]:
# add your code here - consider creating a new cell for each section of code
knn = NearestNeighbors(n_neighbors = 6, metric = 'cosine')
knn.fit(rating_matrix)

NearestNeighbors(metric='cosine', n_neighbors=6)

In [7]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  title_id = filtered_rating_df['title_code'][filtered_rating_df['title'] == book].values[0]
  
  recommended_books = [book]
  pred_query = knn.kneighbors(rating_matrix[title_id])
  nearest_titles = [[filtered_rating_df['title'][filtered_rating_df['title_code'] == i].values[0], round(float(n), 2)] for i, n in zip(pred_query[1][0], pred_query[0][0]) if filtered_rating_df['title'][filtered_rating_df['title_code'] == i].values[0] != book]
  
  recommended_books.append(nearest_titles)
  return recommended_books

In [8]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ['The Lovely Bones: A Novel', 'I Know This Much Is True', 'The Surgeon', 'The Weight of Water', 'White Oleander : A Novel']
  recommended_books_dist = [0.72, 0.77, 0.77, 0.77, 0.79]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', 0.72], ['I Know This Much Is True', 0.77], ['The Surgeon', 0.77], ['The Weight of Water', 0.77], ['White Oleander : A Novel', 0.79]]]
You passed the challenge! 🎉🎉🎉🎉🎉
